In [1]:
#!pip install pyspark
#!pip install kaggle
#!pip install wordcloud

In [47]:
import pyspark
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pyspark.sql import functions as func
from pyspark.sql.types import StringType,FloatType
import nltk

In [48]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('profitdata.csv').getOrCreate()
spark

In [50]:
df = spark.read.csv("profitdata.csv", header=True, inferSchema=True) #Loading the data

In [51]:
df.show(10)

+---------+-------+-------+-----------------+-------------------+------------------+-----------------+------+
|OrderDate| Region|Manager|Software Engineer|   Name of Projects|Number of Projects| Number of Hours |Profit|
+---------+-------+-------+-----------------+-------------------+------------------+-----------------+------+
|   1-6-19|   East|   Rama|    Urvesh Parmar|                NLP|                 3|             50.0| 150.0|
|  1-23-19|Central|  Gopal|Chaitanya Shintre| Linear Regression |                 4|             20.0|  80.0|
|   2-9-19|Central|   Sita|     Manav Parmar|Logistic Regression|                 5|             30.0| 150.0|
|  2-26-19|Central|  Geeta|        Heet Gala|                SVM|                 6|             70.0| 420.0|
|  3-15-19|   West|   Rama|  Sushant Saurabh|                KNN|                 7|             80.0| 560.0|
|   4-1-19|   East|  Gopal|Chaitanya Shintre|Logistic Regression|                 4|             30.0| 120.0|
|  4-19-19

In [52]:
df.printSchema() 

root
 |-- OrderDate: string (nullable = true)
 |-- Region: string (nullable = true)
 |-- Manager: string (nullable = true)
 |-- Software Engineer: string (nullable = true)
 |-- Name of Projects: string (nullable = true)
 |-- Number of Projects: integer (nullable = true)
 |--  Number of Hours : double (nullable = true)
 |-- Profit: double (nullable = true)



In [53]:
df.head(5)

[Row(OrderDate='1-6-19', Region='East', Manager='Rama', Software Engineer='Urvesh Parmar', Name of Projects='NLP', Number of Projects=3,  Number of Hours =50.0, Profit=150.0),
 Row(OrderDate='1-23-19', Region='Central', Manager='Gopal', Software Engineer='Chaitanya Shintre', Name of Projects='Linear Regression ', Number of Projects=4,  Number of Hours =20.0, Profit=80.0),
 Row(OrderDate='2-9-19', Region='Central', Manager='Sita', Software Engineer='Manav Parmar', Name of Projects='Logistic Regression', Number of Projects=5,  Number of Hours =30.0, Profit=150.0),
 Row(OrderDate='2-26-19', Region='Central', Manager='Geeta', Software Engineer='Heet Gala', Name of Projects='SVM', Number of Projects=6,  Number of Hours =70.0, Profit=420.0),
 Row(OrderDate='3-15-19', Region='West', Manager='Rama', Software Engineer='Sushant Saurabh', Name of Projects='KNN', Number of Projects=7,  Number of Hours =80.0, Profit=560.0)]

In [54]:
from pyspark.sql.functions import concat, when, row_number, to_date, year, month, dayofweek
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col

In [55]:
# Filter Data
df.filter(df[" Number of Hours "] > 20).show(10)

+---------+-------+-------+-----------------+-------------------+------------------+-----------------+------+
|OrderDate| Region|Manager|Software Engineer|   Name of Projects|Number of Projects| Number of Hours |Profit|
+---------+-------+-------+-----------------+-------------------+------------------+-----------------+------+
|   1-6-19|   East|   Rama|    Urvesh Parmar|                NLP|                 3|             50.0| 150.0|
|   2-9-19|Central|   Sita|     Manav Parmar|Logistic Regression|                 5|             30.0| 150.0|
|  2-26-19|Central|  Geeta|        Heet Gala|                SVM|                 6|             70.0| 420.0|
|  3-15-19|   West|   Rama|  Sushant Saurabh|                KNN|                 7|             80.0| 560.0|
|   4-1-19|   East|  Gopal|Chaitanya Shintre|Logistic Regression|                 4|             30.0| 120.0|
|  4-19-19|Central|   Sita|        Heet Gala|                NLP|                 6|             50.0| 300.0|
|   5-5-19

In [56]:
# Grouping and Aggregating Data
df.groupBy("Manager").agg({"Number of Projects" : "sum"}).show(5)

+-------+-----------------------+
|Manager|sum(Number of Projects)|
+-------+-----------------------+
|   NULL|                   NULL|
|   Rama|                     36|
|   Sita|                     67|
|  Gopal|                     79|
|  Geeta|                     63|
+-------+-----------------------+



In [57]:
# Grouping and Aggregating Data
df.groupBy("Region").agg({"Profit": "mean"}).show(5)

+-------+------------------+
| Region|       avg(Profit)|
+-------+------------------+
|   NULL|              NULL|
|Central| 337.0833333333333|
|   East|315.38461538461536|
|   West| 378.3333333333333|
+-------+------------------+



In [58]:
# Handling Missing Values
df.fillna(0, subset=["Number of Projects"])

DataFrame[OrderDate: string, Region: string, Manager: string, Software Engineer: string, Name of Projects: string, Number of Projects: int,  Number of Hours : double, Profit: double]

In [59]:
# Sorting Data
df.orderBy("Region").show(5)

+---------+-------+-------+-----------------+-------------------+------------------+-----------------+------+
|OrderDate| Region|Manager|Software Engineer|   Name of Projects|Number of Projects| Number of Hours |Profit|
+---------+-------+-------+-----------------+-------------------+------------------+-----------------+------+
|     NULL|   NULL|   NULL|             NULL|               NULL|              NULL|             NULL|  NULL|
|     NULL|   NULL|   NULL|             NULL|               NULL|              NULL|             NULL|  NULL|
|  4-19-19|Central|   Sita|        Heet Gala|                NLP|                 6|             50.0| 300.0|
|  1-23-19|Central|  Gopal|Chaitanya Shintre| Linear Regression |                 4|             20.0|  80.0|
|   2-9-19|Central|   Sita|     Manav Parmar|Logistic Regression|                 5|             30.0| 150.0|
+---------+-------+-------+-----------------+-------------------+------------------+-----------------+------+
only showi

In [60]:
# Renaming Columns
#df = df.withColumnRenamed("Software Engineer", "Gender").show(5)

In [61]:
# Concatenating Columns
df = df.withColumn("concatenated_column", concat(df["Number of Projects"], df[" Number of Hours "])).show(5)

+---------+-------+-------+-----------------+-------------------+------------------+-----------------+------+-------------------+
|OrderDate| Region|Manager|Software Engineer|   Name of Projects|Number of Projects| Number of Hours |Profit|concatenated_column|
+---------+-------+-------+-----------------+-------------------+------------------+-----------------+------+-------------------+
|   1-6-19|   East|   Rama|    Urvesh Parmar|                NLP|                 3|             50.0| 150.0|              350.0|
|  1-23-19|Central|  Gopal|Chaitanya Shintre| Linear Regression |                 4|             20.0|  80.0|              420.0|
|   2-9-19|Central|   Sita|     Manav Parmar|Logistic Regression|                 5|             30.0| 150.0|              530.0|
|  2-26-19|Central|  Geeta|        Heet Gala|                SVM|                 6|             70.0| 420.0|              670.0|
|  3-15-19|   West|   Rama|  Sushant Saurabh|                KNN|                 7|      

In [62]:
# Dropping Columns
#df = df.drop("concatenated_column")

In [65]:
# Distinct Values in a Column
df.select("Manager").distinct().show()

AttributeError: 'NoneType' object has no attribute 'select'

In [66]:
# Pivot Tables
pivot_df = df.groupBy("Profit").pivot("Software Engineer").agg({"Name of Projects": "sum"})

AttributeError: 'NoneType' object has no attribute 'groupBy'